In [49]:
import numpy as np
from sklearn.linear_model import Lasso
from CoRT_builder import CoRT
import utils
import importlib
import matplotlib.pyplot as plt
from mpmath import mp
importlib.reload(utils)
import warnings
from sklearn.exceptions import ConvergenceWarning


In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
# import os
# print("Thư mục hiện tại:", os.getcwd())
# print("File utils nằm ở:", utils.__file__)
# # CoRT_builder là module chứa class CoRT, ta cần import module để check file
# import CoRT_builder
# print("File CoRT_builder nằm ở:", CoRT_builder.__file__)  

In [52]:
# # OVER CONDITIONING
# n_target = 100
# n_source = 200
# p = 200
# K = 5
# Ka = 3
# h = 30
# lamda = 0.05
# # s_vector = [0.4, 0.4, 0.4, 0.5, 0.5, 0.5, -0.6, -0.6, -0.6, -0.6]
# s_vector = [0,0,0,0,0,0,0,0,0,0]
# T = 5
# s = len(s_vector)
# iteration = 10
# CoRT_model = CoRT(alpha=lamda)
# p_value_list = []

# # target_data, source_data = CoRT.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
# # similar_source_index = CoRT.find_similar_source(n_target, K, target_data, source_data, T=T, verbose=False)
# # print(target_data)
# # print(similar_source_index)

# total_false_positives_detected = 0
# total_false_positives_rejected = 0

# for iter in range(iteration):
#     print(f"Processing iteration {iter+1}")

#     target_data, source_data = CoRT_model.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
#     similar_source_index = CoRT_model.find_similar_source(n_target, K, target_data, source_data, T=T, verbose=False)

#     X_combined, y_combined = CoRT_model.prepare_CoRT_data(similar_source_index, source_data, target_data)

#     model = Lasso(alpha=lamda, fit_intercept=False, random_state=42)
#     model.fit(X_combined, y_combined.ravel())
#     beta_hat_target = model.coef_[-p:]

#     active_indices = np.array([i for i, b in enumerate(beta_hat_target) if b != 0])

#     if len(active_indices) == 0:
#       print(f"Iteration {iter}: Lasso selected no features. Skipping.")
#       continue

#     j = np.random.choice(len(active_indices))

#     X_target = target_data["X"]
#     y_target = target_data["y"]
#     X_active, X_inactive = utils.get_active_X(beta_hat_target, X_target)
#     etaj, etajTy = utils.construct_test_statistic(y_target, j, X_active)

#     Sigma = np.eye(n_target)
#     b_global = Sigma @ etaj @ np.linalg.pinv(etaj.T @ Sigma @ etaj)
#     a_global = (Sigma - b_global @ etaj.T) @ y_target

#     folds = utils.split_target(T, X_target, y_target, n_target)

#     L_train, R_train = utils.get_Z_train(etajTy, folds, source_data, a_global, b_global, lamda, K, T)
#     L_val, R_val = utils.get_Z_val(folds, T, K, a_global, b_global, etajTy, lamda, source_data)
#     L_CoRT, R_CoRT = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, etajTy)

#     L_final, R_final = utils.combine_Z(L_train, R_train, L_val, R_val, L_CoRT, R_CoRT)

#     etaT_sigma_eta = (etaj.T @ Sigma @ etaj).item()
#     sigma_z = np.sqrt(etaT_sigma_eta)
#     truncated_cdf = utils.computed_truncated_cdf(L_final, R_final, etajTy, 0, sigma_z)
#     p_value = 2 * min(truncated_cdf, 1 - truncated_cdf)
#     if p_value is not None:
#       total_false_positives_detected += 1
#       if p_value <= lamda:
#           total_false_positives_rejected += 1
#     p_value_list.append(p_value.item())

# FPR = 0
# if total_false_positives_detected > 0:
#     FPR = total_false_positives_rejected / total_false_positives_detected

# print(FPR, "gay")
# plt.hist(p_value_list, bins=10, density=True, edgecolor='black')
# plt.title("Histogram of p_values")
# plt.xlabel("p_value")
# plt.ylabel("Density")
# plt.show()

In [ ]:
#PARAMETRIC PROGRAMMING

n_target = 20
n_source = 10
p = 40
K = 5
Ka = 3
h = 30
lamda = 0.03
s_vector = [0,0,0,0,0,0,0,0,0,0]
T = 5
s = len(s_vector)
CoRT_model = CoRT(alpha=lamda)
p_values = []

iteration = 100

for step in range(iteration):
    target_data, source_data = CoRT_model.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
    similar_source_index = CoRT_model.find_similar_source(n_target, K, target_data, source_data, T=T, verbose=False)
    X_combined, y_combined = CoRT_model.prepare_CoRT_data(similar_source_index, source_data, target_data)

    model = Lasso(alpha=lamda, fit_intercept=False, tol=1e-6, max_iter=500000)
    model.fit(X_combined, y_combined.ravel())
    beta_hat_target = model.coef_[-p:]

    active_indices = np.array([i for i, b in enumerate(beta_hat_target) if b != 0])
    if len(active_indices) == 0:
        print(f"Iteration {iter}: Lasso selected no features. Skipping.")
        continue

    j = np.random.choice(len(active_indices))
    X_target = target_data["X"]
    y_target = target_data["y"]
    X_active, X_inactive = utils.get_active_X(beta_hat_target, X_target)
    etaj, etajTy = utils.construct_test_statistic(y_target, j, X_active)

    Sigma = np.eye(n_target)
    b_global = Sigma @ etaj @ np.linalg.pinv(etaj.T @ Sigma @ etaj)
    a_global = (Sigma - b_global @ etaj.T) @ y_target
    folds = utils.split_target(T, X_target, y_target, n_target)

    z_k = -20
    z_max = 20
    Z_train_list = utils.get_Z_train(z_k, folds, source_data, a_global, b_global, lamda, K, T)
    Z_val_list = utils.get_Z_val(z_k, folds, T, K, a_global, b_global, lamda, source_data)
    L_CoRT, R_CoRT, Az = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, z_k)
    offset = p * len(similar_source_index)
    Az_target_only = np.array([idx - offset for idx in Az if idx >= offset])
    z_list = [z_k]
    Az_list = []
    while z_k < z_max:
        Az_target_current = np.array([idx - offset for idx in Az if idx >= offset])
        Az_list.append(Az_target_current)
        for val in Z_train_list:
            if val[4] <= z_k:
                l, r = utils.update_Z_train(val, z_k, folds, source_data, a_global, b_global, lamda, K, T)
                val[3] = l
                val[4] = r
        for val in Z_val_list:
            if val[3] <= z_k:
                l, r = utils.update_Z_val(val, z_k, folds, T, K, a_global, b_global, lamda, source_data)
                val[2] = l
                val[3] = r
        if R_CoRT <= z_k:
            L_CoRT, R_CoRT, Az = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, z_k)
        mn = z_max
        for val in Z_train_list:
            if mn > val[4]:
                mn = val[4]
        for val in Z_val_list:
            if mn > val[3]:
                mn = val[3]
        if mn > R_CoRT:
            mn = R_CoRT
        R_final = mn
        # print(f"[{z_k}] -> {mx, mn}")
        z_k = max(R_final, z_k) + 0.001
        # print(f"Processing {z_k}")
        # print(z_k)
        if (z_k >= z_max):
            z_list.append(z_max)
        else:
            z_list.append(z_k)
    pivot_value = utils.pivot(active_indices, Az_list, z_list, etaj, etajTy, 0, Sigma)
    p_values.append(pivot_value)
    print(f"Processingggggggg {step, pivot_value}")

# Plot results
plt.hist(p_values, density=True, bins=30, edgecolor='black')
plt.title("p-value Distribution")
plt.xlabel("p-value")
plt.ylabel('Density')
plt.tight_layout()
plt.show()
    # etaT_sigma_eta = (etaj.T @ Sigma @ etaj).item()
    # sigma_z = np.sqrt(etaT_sigma_eta)
    # truncated_cdf = utils.computed_truncated_cdf(L_final, R_final, etajTy, 0, sigma_z)
    # p_value = 2 * min(truncated_cdf, 1 - truncated_cdf)

SyntaxError: expected ':' (2783006947.py, line 65)

In [ ]:
# #PARAMETRIC PROGRAMMING

# n_target = 20
# n_source = 10
# p = 40
# K = 5
# Ka = 3
# h = 30
# lamda = 0.03
# s_vector = [0,0,0,0,0,0,0,0,0,0]
# T = 5  
# s = len(s_vector)
# CoRT_model = CoRT(alpha=lamda)
# p_values = []

# iteration = 10

# for step in range(iteration):
#     target_data, source_data = CoRT_model.gen_data(n_target, n_source, p, K, Ka, h, s_vector, s, "AR")
#     similar_source_index = CoRT_model.find_similar_source(n_target, K, target_data, source_data, T=T, verbose=False)
#     X_combined, y_combined = CoRT_model.prepare_CoRT_data(similar_source_index, source_data, target_data)

#     model = Lasso(alpha=lamda, fit_intercept=False, tol=1e-8, max_iter=10000000)
#     model.fit(X_combined, y_combined.ravel())
#     beta_hat_target = model.coef_[-p:]

#     active_indices = np.array([i for i, b in enumerate(beta_hat_target) if b != 0])
#     if len(active_indices) == 0:
#         print(f"Iteration {iter}: Lasso selected no features. Skipping.")
#         continue

#     j = np.random.choice(len(active_indices))
#     X_target = target_data["X"]
#     y_target = target_data["y"]
#     X_active, X_inactive = utils.get_active_X(beta_hat_target, X_target)
#     etaj, etajTy = utils.construct_test_statistic(y_target, j, X_active)

#     Sigma = np.eye(n_target)
#     b_global = Sigma @ etaj @ np.linalg.pinv(etaj.T @ Sigma @ etaj)
#     a_global = (Sigma - b_global @ etaj.T) @ y_target
#     folds = utils.split_target(T, X_target, y_target, n_target)

#     z_k = -20
#     z_max = 20
    
#     z_list = [z_k]
#     Az_list = []
#     while z_k < z_max:
#         mn = z_max
#         Z_train_list = utils.get_Z_train(z_k, folds, source_data, a_global, b_global, lamda, K, T)
#         Z_val_list = utils.get_Z_val(z_k, folds, T, K, a_global, b_global, lamda, source_data)
#         L_CoRT, R_CoRT, Az = utils.get_Z_CoRT(X_combined, similar_source_index, lamda, a_global, b_global, source_data, z_k)
#         offset = p * len(similar_source_index)
#         Az_target_current = np.array([idx - offset for idx in Az if idx >= offset])
#         Az_list.append(Az_target_current)
#         print(z_k)
#         if np.array_equal(active_indices, Az_target_current):
#             print(f"Dit con me may Hiep {step}")
#         for val in Z_train_list:
#             mn = min(mn, val[4])
#         for val in Z_val_list:
#             mn = min(mn, val[3])
#         mn = min(mn, R_CoRT)
#         R_final = mn
#         z_k = R_final + 0.01
#         if (z_k >= z_max):
#             z_list.append(z_max)
#         else:
#             z_list.append(z_k)
#         # print(f"Processing {z_k}")

#     pivot_value = utils.pivot(active_indices, Az_list, z_list, etaj, etajTy, 0, Sigma)
#     p_values.append(pivot_value)
#     print(f"Processing {step, pivot_value}")

# # Plot results
# plt.hist(p_values, density=True, bins=30, edgecolor='black')
# plt.title("p-value Distribution")
# plt.xlabel("p-value")
# plt.ylabel('Density')
# plt.tight_layout()
# plt.show()
#     # etaT_sigma_eta = (etaj.T @ Sigma @ etaj).item()
#     # sigma_z = np.sqrt(etaT_sigma_eta)
#     # truncated_cdf = utils.computed_truncated_cdf(L_final, R_final, etajTy, 0, sigma_z)
#     # p_value = 2 * min(truncated_cdf, 1 - truncated_cdf)

-20
-19.977075325557724
-19.96677969144343
-19.94816999930398
-19.91821077638241
-19.836963088167114
-19.795253581985218
-19.735575872166304
-19.713655267160632
-19.627850305625373
-19.577300396345862
-19.56285571927003
-19.513852248027273
-19.485827606873823
-19.337161313946094
-19.314966465025076
-19.289149185907043
-19.26802538880274
-19.221449898365595
-19.08209283579811
-19.054388056848893
-19.033029819896374
-19.020809063562126
-18.996957420521785
-18.97623942892628
-18.929078003770886
-18.863524981995507
-18.71885249704919
-18.689572116966385
-18.57855251945182
-18.55022298583267
-18.536325994761633
-18.50210853579362
-18.29773691698114
-18.25951895667067
-18.200864531367323
-18.132356785298057
-18.077577572485463
-18.05007353654734
-18.020454446697496
-17.99784465456932
-17.934463655558854
-17.879931456522105
-17.865611913609403
-17.847987890967122
-17.756400886775204
-17.729061442430332
-17.691766651684834
-17.680630671153338
-17.656548074921904
-17.583480683314036
-17.5148894

KeyboardInterrupt: 